In [1]:
!pip install atari-py

In [1]:
import gym 
import random

In [2]:
env = gym.make('SpaceInvaders-v0')
#env = gym.make('Breakout-v0')
states = env.observation_space.shape
#height, width, channels = env.observation_space.shape
actions = env.action_space.n
print(states)

(210, 160, 3)


In [3]:
print(env.unwrapped.get_action_meanings())
lst = []
for i in range(len(env.unwrapped.get_action_meanings())):
    lst.append(i)
print(lst)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']
[0, 1, 2, 3, 4, 5]


In [5]:
episodes = 4
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice(lst)
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:210.0
Episode:2 Score:110.0
Episode:3 Score:165.0
Episode:4 Score:415.0


In [72]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [62]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    #model.add(Dense(512, activation='relu'))
    #model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [63]:
model = build_model(height, width, channels, actions)

In [64]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 405510    
Total params: 481,446
Trainable params: 481,446
Non-trainable params: 0
_________________________________________________________________


In [14]:
del model

In [65]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [66]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [67]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3))

In [68]:
dqn.fit(env, nb_steps=3000, visualize=False, verbose=2)

Training for 3000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  947/3000: episode: 1, duration: 17.036s, episode steps: 947, steps per second:  56, episode reward: 225.000, mean reward:  0.238 [ 0.000, 30.000], mean action: 2.569 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
done, took 146.793 seconds
